# Simulation in R

In this document we will explore ways to run simulations with R, and use some random number generators.


We can generate random numbers using a variety of functions and distributions. 

The first kind of randomness that people understand is uniform randomness. This is where everything is equally likely. We generate randomness like this with ```runif(min, max)```.

```
library(dplyr)
library(ggplot2)
runif(n=10, min=0, max=1)
```

In [0]:
library(dplyr)
library(ggplot2)
runif(n=10, min=0, max=1)

Now let's save some random numbers and make a table.

In [0]:
samp<-runif(n=1000000, min=0, max=1)

In [0]:
num<-c(1:1000000)

In [0]:
table<-data.frame(cbind(num, samp))

In [0]:
head(table)

In [0]:
summary(table)

## How to use random numbers

We might be trying to simualate a batter in a baseball game. Maybe we are trying to simulate Mike Trout. He is one of the best hitter in baseball today. His batting average is 0.312. So we could say that he gets a hit if the random number is less than .312. This is easily done with the ```mutate``` command.

```
table <- table %>%
mutate(hit = ifelse(samp < .312, 1, 0))
head(table)
```

In [0]:
table <- table %>%
mutate(hit = ifelse(samp < .312, 1, 0))
head(table)

# Num of hits

We can calculate the number of hits by ```sum(table$hit)```

In [0]:
sum(table$hit)


## Seeds and random number generators

Computers don't actually create randomness. They actually use complicated algorithms that produce a determined number. These numbers just look random to use. We can make all the computers in this room come up with the same random numbers by setting the *seed* on the computers to be the same. 

```
set.seed(13)
runif(n=10, min=0, max=1)
```

Compare your numbers to another classmates.

In [0]:
set.seed(13)
runif(n=10, min=0, max=1)

In [0]:
set.seed(13)
runif(n=10, min=0, max=1)

## Evaluating streakiness

Are players streaky? Or is it just the whims of fate. Let's look at some data and compare it to what we see if the world was just mathematically random.


In [0]:
load(file="kobe_basket.rda")

In [0]:
kobe_basket

**Warning: Don't wory about the next line. It is a function that we need to determine how streaky Kobe is.**

In [0]:
calc_streak = function(x)
{
    if (!is.atomic(x))
        x = x[,1]

    if (any(!x %in% c("H","M")))
        stop('Input should only contain hits ("H") and misses ("M")')
    
    y = rep(0,length(x))
    y[x == "H"] = 1
    y = c(0, y, 0)
    wz = which(y == 0)
    streak = diff(wz) - 1
    
    return(data.frame(length = streak))
}

In [0]:
dim(kobe_basket)

In [0]:
kobe_streak <- calc_streak(kobe_basket$shot)
IQR(kobe_streak$length)

In [0]:
ggplot(data = kobe_streak, aes(x = length)) +
  geom_histogram(binwidth = 1)

**Describe Kobe Bryants streak distribution.**

## Simulated Kobe

Kobe was about 45% shooter. So we want to imagine a shooter that isn't streaky at all that just works the way randomness says the situation should work. Check out the code below.

In [0]:
shot_outcomes <- c("H", "M")

Now ```shot_outcomes``` is a list of two outcomes H and M for hit and miss. 

We now ```sample``` from this list the same number of times that Kobe shot in his game, 133. 

Here we use the ```sample``` function this allows us to imagine picking from a list over and over. There are various options to the command as well, as you will see.

In [0]:
sim_basket <- sample(shot_outcomes, size = 133, replace = TRUE, prob=c(0.45, 0.55))
sim_basket

This creates a list of hits and misses with the same expected proportions as a typical Kobe Bryant after 133 shots.


In [0]:

sim_streak = calc_streak(sim_basket)
ggplot(data = sim_streak, aes(x=length)) +
  geom_histogram(binwidth = 1)

In [0]:
IQR(sim_streak$length)

In [0]:
streaks = c(kobe_streak,sim_streak)
boxplot(streaks)

**What does this say about Kobe being a streaky shooter?**

## Simulating stock market

Stock returns are much more likely to follow the normal model than a uniform model. A fictional stock has the following properties: On average it gains 1.001 times its opening price during the trading day, but that can vary by a standard deviation of 0.005 on any given day (this is its volatility). We can simulate a single sample path for this stock by taking the cumulative product from a Normal distribution with a mean of 1.001 and a sd of 0.005. Assuming the stock opens at $20/per share, here is a sample path for 365 days of trading.



In [0]:
days <- 365
changes <- rnorm(365,mean=1.001,sd=0.005)
plot(cumprod(c(20,changes)),type='l',ylab="Price",xlab="day",main="closing price (1 possible path)")

In [0]:
runs <- 100000
#simulates future movements and returns the closing price on day 365
generate.path <- function(){
  days <- 365
  changes <- rnorm(365,mean=1.001,sd=0.005)
  sample.path <- cumprod(c(20,changes))
  closing.price <- sample.path[days+1] #+1 because we add the opening price
}

mc.closing <- replicate(runs,generate.path())

In [0]:
head(mc.closing)

In [0]:
mean(mc.closing)

In [0]:
quantile(mc.closing, c(.05, .5, .95))


### What would be a big streak for Kobe?

We again come back to Kobe, a player we know makes 45% percent of his shots. Kobe took about 26500 shots in his career. What would be a reasonable longest shooting streak for a player like that?


**Run a simulation of Kobe's career and see what the longest streak was? Then do it again. Did you get a different answer? Do it a few times. How much variability is there in the answer?**